In [ ]:
import requests
import pandas as pd
import os  
import time
import urllib.request
from urllib.request import urlopen
from urllib.error import URLError, HTTPError
from urllib.parse import urlparse

In [ ]:
term = "bouguereau"

In [ ]:
#Make request
api_url = "https://collectionapi.metmuseum.org/public/collection/v1"
endpoint = "/search"
url = api_url+endpoint
#Dictionary of parameters – check the documentation for parameters to your liking
params = {'q': term, 
          'hasImages': True,
          'departmentID': 11, #in the documentation I found that the ID for European Paintings is 11
          # 'dateBegin' : 1800,
          # 'dateEnd': 1905
          }
response = requests.get(url, params = params)
print(response.url)
print("STATUS:", response.status_code)

In [ ]:
temp_json = response.json()
temp_json

In [ ]:
results = []

api_url = "https://collectionapi.metmuseum.org/public/collection/v1/objects/"

for i in temp_json['objectIDs']:
    url = api_url + str(i)
    response = requests.get(url).json()
    results.append(response)

print(len(results))

In [ ]:
#using pandas, we can use json_normalize to put our data into a dataframe!
df = pd.json_normalize(results)

df = df[['objectID', 'primaryImage','department','objectName','title','artistDisplayName','objectDate']]

# df = df[df['primaryImage'].notna()]
df

In [ ]:

def merge(list1, list2):    
    merged_list = [(list1[i], list2[i]) for i in range(0, len(list1))]
    return merged_list

def download_image(img_list, file_path):
    # print(image_list[1])
    # img_list[1].decode("utf-8")
    filename = '{}.jpg'.format(img_list[0])
    fullpath = '{}{}'.format(file_path, filename)
    try: 
        urllib.request.urlretrieve(img_list[1], fullpath) 
    except URLError:
        print(f"Error downloading: {img_list[0]}")
        errorlist.append(img_list[0])
    except HTTPError:
        print(f"Error downloading: {img_list[0]}")
        errorlist.append(img_list[0])

In [ ]:
#this returns a list of tuples of object ID and their image URls
image_url = df['primaryImage'].to_list()

# image_url = image_url.replace("\\", "/")
image_url = [str(str(i).replace("\\", "/").replace(" ", "")) for i in image_url]

image_id = df['objectID'].to_list()
image_list = merge(image_id, image_url)

In [ ]:
print(image_url[6])

In [ ]:
directory = term
file_path = os.path.join("data/", directory + "/")  
os.mkdir(file_path)  

In [ ]:


#download images, BE CAREFUL about how many images you have! i'd suggest to stop at 100. simply index image_list like so: image_list[:n]

#the errorlist flags what images did not download for whatever reason
errorlist= []

t1 = time.perf_counter()

for url in image_list:
    # url.decode('utf-8')
    try:
         download_image(url, file_path)
    except UnicodeEncodeError:
        print("probably has a - somewhere")
    except ValueError:
        print("no link for some reason")
    
        
t2 = time.perf_counter()

print(f'Finished in {t2-t1} seconds')